In [7]:
import sys
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
import sklearn
import xgboost
import lightgbm
# import category_encoders
import torch
import torchvision
import torchaudio
import optuna
import pytorch_lightning
import imblearn
import tensorboard

# Print versions
print("✅ Installed Package Versions:")
print(f"Python               : {sys.version.split()[0]}")
print(f"Pandas               : {pd.__version__}")
print(f"Numpy                : {np.__version__}")
print(f"Matplotlib           : {matplotlib.__version__}")
print(f"Seaborn              : {sns.__version__}")
print(f"Scikit-learn         : {sklearn.__version__}")
print(f"XGBoost              : {xgboost.__version__}")
print(f"LightGBM             : {lightgbm.__version__}")
# print(f"Category Encoders    : {category_encoders.__version__}")
print(f"Imbalanced-learn     : {imblearn.__version__}")
print(f"Torch                : {torch.__version__}")
print(f"Torchvision          : {torchvision.__version__}")
print(f"Torchaudio           : {torchaudio.__version__}")
print(f"Optuna               : {optuna.__version__}")
print(f"PyTorch Lightning    : {pytorch_lightning.__version__}")
print(f"TensorBoard          : {tensorboard.__version__}")


✅ Installed Package Versions:
Python               : 3.12.9
Pandas               : 2.3.0
Numpy                : 2.3.1
Matplotlib           : 3.10.3
Seaborn              : 0.13.2
Scikit-learn         : 1.6.1
XGBoost              : 3.0.2
LightGBM             : 4.6.0
Imbalanced-learn     : 0.13.0
Torch                : 2.7.1+cpu
Torchvision          : 0.22.1+cpu
Torchaudio           : 2.7.1+cpu
Optuna               : 4.4.0
PyTorch Lightning    : 2.5.2
TensorBoard          : 2.19.0


In [6]:
2+2

import torch
from torch import nn

Demo JUst coppied from net 

Pytorch+ sklearn starter pack

In [8]:
# train.py

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import ReduceLROnPlateau
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

# =============== Config ===============
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-3

# =============== Dataset Class ===============
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# =============== Simple ANN Model ===============
class ANN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.model(x)

# =============== Load and preprocess data ===============
df = pd.read_csv("data/dataset.csv")  # Update path

# Encode categorical features
df = df.dropna()  # Drop missing for simplicity (you can use KNNImputer)
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

# Features and target
X = df.drop("target", axis=1).values  # Change 'target' to your column
y = df["target"].values

# Optional: handle imbalance
X, y = SMOTE().fit_resample(X, y)

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Dataset and Dataloader
train_ds = TabularDataset(X_train, y_train)
val_ds = TabularDataset(X_val, y_val)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)

# =============== Model, Loss, Optimizer ===============
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ANN(input_dim=X_train.shape[1], output_dim=len(np.unique(y))).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# =============== Training Loop ===============
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            y_pred.extend(preds)
            y_true.extend(y_batch.numpy())

    val_f1 = f1_score(y_true, y_pred, average='macro')
    val_acc = accuracy_score(y_true, y_pred)
    scheduler.step(1 - val_f1)

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss:.4f}, Val F1: {val_f1:.4f}, Acc: {val_acc:.4f}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/dataset.csv'